In [37]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
from keras.layers import Lambda
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input as resnet50_pre
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input as inceptionV3_pre
from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input as xception_pre
from keras.preprocessing import image                  
from tqdm import tqdm
from PIL import ImageFile  
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense, Input
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint  
import numpy as np
import pandas as pd
from glob import glob
import cv2                
import matplotlib.pyplot as plt    
import matplotlib.image as mpimg
%matplotlib inline 
import random
import os
import shutil
import h5py

## 对图片进行分类

In [3]:

train_path = 'dogs-vs-cats/train1/'
test_path = 'dogs-vs-cats/test1/'

dog_path = train_path + 'dog/'
cat_path = train_path + 'cat/'

if not os.path.exists(dog_path):
    names= os.listdir(train_path)
    os.mkdir(dog_path)
    os.mkdir(cat_path)
    [shutil.move(train_path + name, cat_path + name) for name in names if name.startswith('cat')]
    [shutil.move(train_path + name, dog_path + name) for name in names if name.startswith('dog')]
    


In [28]:
def write_gap(MODEL, image_size, lambda_func=None):
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)

    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
    #进行这一步时会从Keras网站中下载imagenet模型，如果网速慢或缺乏连接，
    #可以直接在上面的连接下载，放到C:\Users\用户名\.keras\models 下
    #（模型有可能不是最新的，导致进行到这一步仍然报错），
    #或者直接点击报错的GitHub链接进行下载
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))

    gen = ImageDataGenerator()
    train_path = "dogs-vs-cats/train1/"
    test_path = 'dogs-vs-cats/test1/'
    train_generator = gen.flow_from_directory(train_path, image_size, shuffle=False,
                                              batch_size=1)
    test_generator = gen.flow_from_directory(test_path, image_size, shuffle=False,
                                             batch_size=1, class_mode=None)

    train = model.predict_generator(train_generator, train_generator.samples)
    test = model.predict_generator(test_generator, test_generator.samples)
    with h5py.File("gap_{0}.h5".format(base_model.name)) as h:
        h.create_dataset("train", data=train)
        h.create_dataset("test", data=test)
        h.create_dataset("label", data=train_generator.classes)

In [17]:
write_gap(ResNet50, (224, 224))


C:\ProgramData\Anaconda3\envs\graduate\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Found 200 images belonging to 2 classes.
Found 100 images belonging to 1 classes.


In [29]:
write_gap(InceptionV3, (299, 299), inceptionV3_pre)

Found 200 images belonging to 2 classes.
Found 100 images belonging to 1 classes.


In [30]:
write_gap(Xception, (299, 299), xception_pre)

Found 200 images belonging to 2 classes.
Found 100 images belonging to 1 classes.


In [31]:
np.random.seed(2017)

X_train = []
X_test = []
for filenames in ["gap_resnet50.h5", "gap_inception_v3.h5", "gap_xception.h5"]:
    filename = filenames
    with h5py.File(filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['label'])
X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)


In [32]:
inputs = Input(X_train.shape[1:])#shape=（2048*3，）
x = Dropout(0.5)(inputs)
x = Dense(1, activation='sigmoid')(x)
model = Model(inputs, x)
model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])


W0318 20:00:46.500364 14516 deprecation.py:506] From C:\ProgramData\Anaconda3\envs\graduate\lib\site-packages\keras\backend\tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [34]:
model.fit(X_train, y_train, batch_size=128, nb_epoch=8, validation_split=0.2,verbose=2)

y_pred = model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)

C:\ProgramData\Anaconda3\envs\graduate\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Train on 160 samples, validate on 40 samples
Epoch 1/1
 - 6s - loss: 0.6713 - acc: 0.6063 - val_loss: 0.3488 - val_acc: 0.8750
100/100 [==============================] - ETA:  - 5s 47ms/step


In [39]:
df = pd.read_csv("dogs-vs-cats/sample_submission.csv")

gen = ImageDataGenerator()
test_path = 'dogs-vs-cats/test1/'
test_generator = gen.flow_from_directory(test_path, (224, 224), shuffle=False,
                                         batch_size=1, class_mode=None)

for i, fname in enumerate(test_generator.filenames):
    index = int(fname[fname.rfind('\\')+1:fname.rfind('.')])
    df.set_value(index-1, 'label', y_pred[i])

df.to_csv('pred.csv', index=None)
df.head(10)


Found 100 images belonging to 1 classes.


C:\ProgramData\Anaconda3\envs\graduate\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.


,id,label
0,1,0.670915
1,2,0.688015
2,3,0.519156
3,4,0.686481
4,5,0.127145
5,6,0.158234
6,7,0.193827
7,8,0.265775
8,9,0.179208
9,10,0.322614
